<a href="https://colab.research.google.com/github/MortenWillendrup/Thesis/blob/master/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading data and defining functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MBSPrepaymentPrediction')
os.getcwd()
ROOT_DIR = r'/content/drive/MyDrive/MBSPrepaymentPrediction'

In [ ]:
import os
import math
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#from Definitions import ROOT_DIR


class StandardLoanLevelDatasetParser:

    def __init__(self, max_rows_per_quarter=None, rows_to_sample=None, dump_to_csv=None, seed=None):

        self.sll_dataset_dir = os.path.join(ROOT_DIR, "Datasets/StandardLoanLevelDataset/Data/")
        #print(self.sll_dataset_dir)
        self._issuance_cols = [
            'credit_score', 'first_payment_date', 'first_time_homebuyer_flag', 'maturity_date', 'MSA', 'MI_%',
            'number_of_units', 'occupancy_status', 'orig_CLTV', 'orig_DTI', 'orig_UPB', 'orig_LTV',
            'orig_interest_rate', 'channel', 'PPM', 'amortization_type', 'property_state', 'property_type',
            'postal_code', 'loan_sequence_number', 'loan_purpose', 'orig_loan_term', 'number_of_borrowers',
            'seller_name', 'servicer_name', 'super_conforming', 'pre-harp_sequence_number', 'program_indicator',
            'harp_indicator', 'property_valuation_method', 'io_indicator'
        ]
        self._issuance_string = "historical_data"

        self._performance_cols = [
            'loan_sequence_number', 'report_month', 'current_UPB', 'current_loan_dlqc_status', 'loan_age',
            'months_to_maturity', 'repurchase', 'modification', 'zero_balance_code', 'zero_balance_date',
            'current_interest_rate', 'current_deffered_UPB', 'DDLPI', 'MI_recoveries', 'net_sales_proceeds',
            'non_MI_recoveries', 'expenses', 'legal_costs', 'maintenance_costs', 'taxes_and_insurence',
            'miscellaneous_expenses', 'actual_loss', 'modification_cost', 'step_modification', 'deferred_payment_plan',
            'estimated_LTV', 'zero_balance_removal_UPB', 'dlq_accrued_interest', 'dlqc_due_to_disaster',
            'borrower_assistance_status'
        ]
        self._performance_string = self._issuance_string + "_time"

        self.us_hpa_path = os.path.join(ROOT_DIR, "Datasets/MacroData/US_HPA.csv")
        self.us_hpa_cols = ['hpa_state', 'hpa_year', 'hpa_quarter', 'hpi']

        self.state_hpa_path = os.path.join(ROOT_DIR, "Datasets/MacroData/state_HPA.csv")
        self.state_hpa_cols = ['hpa_state', 'hpa_year', 'hpa_quarter', 'hpi']

        self.mtg_rate_path = os.path.join(ROOT_DIR, "Datasets/MacroData/mortgage_rate.csv")
        self.mtg_rate_cols = ['mtg_date', 'mtg_rate']
        self.mtg_rate_header = 8

        self.unemployment_rate_path = os.path.join(ROOT_DIR, "Datasets/MacroData/unemployment_rate.csv")
        self.unemployment_rate_cols = ['unemp_series_id', 'unemp_year', 'unemp_period', 'unemp_rate']
        self.unemployment_rate_header = 0

        self.data_types = {
            # Issuance data types
            'credit_score': 'int64', 'first_payment_date': 'int64', 'first_time_homebuyer_flag': 'S',
            'maturity_date': 'int64', 'MSA': 'float64', 'MI_%': 'S', 'number_of_units': 'S',
            'occupancy_status': 'S', 'orig_CLTV': 'int64', 'orig_DTI': 'int64', 'orig_UPB': 'int64', 'orig_LTV': 'int64',
            'orig_interest_rate': 'S', 'channel': 'S', 'PPM': 'S', 'amortization_type': 'S',
            'property_state': 'S', 'property_type': 'S', 'postal_code': 'int64', 'loan_sequence_number': 'S',
            'loan_purpose': 'S', 'orig_loan_term': 'int64', 'number_of_borrowers': 'int64', 'seller_name': 'S',
            'servicer_name': 'S', 'super_conforming': 'S', 'pre-harp_sequence_number': 'S', 'program_indicator': 'S',
            'harp_indicator': 'S', 'property_valuation_method': 'int64', 'io_indicator': 'S',
            # Performance data types
            'report_month': 'int64', 'current_UPB': 'float64', 'current_loan_dlqc_status': 'S', 'loan_age': 'int64',
            'months_to_maturity': 'int64', 'repurchase': 'S', 'modification': 'S', 'zero_balance_code': 'float64',
            'zero_balance_date': 'float64', 'current_interest_rate': 'float64', 'current_deffered_UPB': 'S',
            'DDLPI': 'float64', 'MI_recoveries': 'float64', 'net_sales_proceeds': 'float64', 'non_MI_recoveries': 'float64',
            'expenses': 'float64', 'legal_costs': 'float64', 'maintenance_costs': 'float64',
            'taxes_and_insurence': 'float64', 'miscellaneous_expenses': 'float64', 'actual_loss': 'float64',
            'modification_cost': 'float64', 'step_modification': 'S', 'deferred_payment_plan': 'S',
            'estimated_LTV': 'float64', 'zero_balance_removal_UPB': 'float64', 'dlq_accrued_interest': 'float64',
            'dlqc_due_to_disaster': 'S', 'borrower_assistance_status': 'S',
            # US / State HPA data types
            'hpa_state': 'S', 'hpa_year': 'int64', 'hpa_quarter': 'int64', 'hpi': 'float64',
            # Mortgage Rate data types
            'mtg_date': 'S', 'mtg_rate': 'float64',
            # Unemployment Rate data types
            'unemp_series_id': 'S', 'unemp_year': 'int64', 'unemp_period': 'S', 'unemp_rate': 'float64'
        }

        self.categorical_cols = ['occupancy_status', 'property_state', 'loan_purpose', 'month']
        self.numerical_cols = [
            'credit_score', 'first_time_homebuyer_flag', 'MI_%', 'number_of_units', 'orig_CLTV', 'orig_DTI', 'orig_UPB',
            'orig_LTV', 'orig_interest_rate', 'PPM', 'number_of_borrowers', 'program_indicator', 'harp_indicator',
            'current_UPB', 'loan_age', 'months_to_maturity', 'modification', 'current_interest_rate',
            'current_deffered_UPB', 'step_modification', 'deferred_payment_plan', 'borrower_assistance_status', 'hpa',
            'mtg_rate', 'unemp_rate', 'zero_balance_code'
        ]


        self.max_rows_per_quarter = max_rows_per_quarter
        self.data = pd.DataFrame()
        self.seed = seed
        self.rows_to_sample = rows_to_sample
        self.dump_to_csv = dump_to_csv

    def load(self):
        print(f"Loading Standard Loan-Level Dataset at path {self.sll_dataset_dir}")
        #print("this is working")
        for root, dirs, _ in os.walk(self.sll_dataset_dir):
            #print(root)
            #print(dirs)
            #print(_)
            for dir_name in _:
                #print(f' this is the dir name {dir_name}')
                # if "historical_data_" not in dir_name or "Q" not in dir_name:
                #     continue
                #print('\n')
                print(f"Adding data from {dir_name} to dataset...")
                dir_path = os.path.join(root, dir_name)

                split_name = dir_path.split('_')
                year = split_name[-1][:4]
                quarter = split_name[-1][4:]

                issuance_path = os.path.join(root, self._issuance_string + "_" + year + quarter )
                #print(issuance_path)
                performance_path = os.path.join(root, self._performance_string + "_" + year + quarter )
                #print(performance_path)
                issuance = pd.read_csv(issuance_path, delimiter='|', names=self._issuance_cols, dtype=self.data_types,
                                       nrows=self.max_rows_per_quarter, error_bad_lines=False)
                issuance = issuance[issuance.eval(
                    "amortization_type=='FRM' & property_type=='SF' & orig_loan_term==360 & io_indicator=='N'")]
                
                #print('issuance loaded')
                if self.rows_to_sample:
                    issuance = issuance.sample(min(self.rows_to_sample, len(issuance.index)), random_state=self.seed)

                performance = pd.read_csv(performance_path, delimiter='|', names=self._performance_cols, dtype=self.data_types, nrows=self.max_rows_per_quarter, index_col=False)
                performance = performance.loc[performance['loan_sequence_number'].isin(set(issuance['loan_sequence_number']))]
                #print('issuance loaded')
                full_data = performance.join(issuance.set_index('loan_sequence_number'), on='loan_sequence_number')
                full_data['year'] = int(year)
                full_data['quarter'] = int(quarter[1])
                full_data['report_year'] = full_data['report_month'] // 100
                full_data['report_quarter'] = full_data['report_month'] % 100 // 4 + 1
                self.data = self.data.append(full_data)

        us_hpa = pd.read_csv(self.us_hpa_path, delimiter=';', names=self.us_hpa_cols, dtype=self.data_types) # setting the US HPI as the HPI for Puerto Rico
        us_hpa = us_hpa[us_hpa['hpa_state'] == 'USA']
        us_hpa['hpa'] = ((us_hpa.hpi / us_hpa.hpi.shift(1)) ** 4 - 1) * 100 # Should use interpolate instead , look at https://stackoverflow.com/questions/61761785/pandas-convert-quarterly-data-into-monthly-data
        us_hpa['hpa_state'] = 'PR' 

        state_hpa = pd.read_csv(self.state_hpa_path, delimiter=',', names=self.state_hpa_cols, dtype=self.data_types)
        state_hpa['hpa'] = ((state_hpa.hpi / state_hpa.hpi.shift(1)) ** 4 - 1) * 100

        hpa = pd.concat([state_hpa, us_hpa])
        self.data = self.data.join(hpa.set_index(['hpa_state', 'hpa_year', 'hpa_quarter']), on=['property_state', 'report_year', 'report_quarter'])

        mtg_rate = pd.read_csv(self.mtg_rate_path, header=self.mtg_rate_header, names=self.mtg_rate_cols, dtype=self.data_types)
        split_date = mtg_rate['mtg_date'].str.split('-', expand=True)
        mtg_rate['yearmon'] = split_date[0].astype(int) * 100 + split_date[1].astype(int)
        mtg_rate = mtg_rate.drop(columns='mtg_date')
        self.data = self.data.join(mtg_rate.set_index('yearmon'), on='report_month')

        unemp = pd.read_csv(self.unemployment_rate_path, delimiter=';', header=self.unemployment_rate_header, names=self.unemployment_rate_cols, dtype=self.data_types)
        unemp.columns = self.unemployment_rate_cols
        unemp['yearmon'] = unemp['unemp_year'] * 100 + unemp['unemp_period'].str.slice(1, 3).astype(int)
        unemp = unemp.drop(columns=['unemp_series_id', 'unemp_year', 'unemp_period'])
        self.data = self.data.join(unemp.set_index('yearmon'), on='report_month')
        #print("time for cleaning")
        self._clean()

        #self.data = self.data[self.use_cols]

        if self.dump_to_csv:
            self.data.to_csv(self.sll_dataset_dir+'/'+dir_name[0:-4]+'.csv', index=False)

    def _clean(self):
        self.data['MSA'] = self.data['MSA'].apply(lambda x: 999 if math.isnan(x) else x)
        self.data['number_of_units'] = self.data['number_of_units'].apply(lambda x: 99 if x == '.' else int(x))
        self.data['harp_indicator'] = self.data['harp_indicator'].apply(lambda x: 0 if x != 'Y' else 1)
        self.data['first_time_homebuyer_flag'] = self.data['first_time_homebuyer_flag'].apply(lambda x: 0 if x == 'N' else 1)
        self.data['program_indicator'] = self.data['program_indicator'].apply(lambda x: 0 if x == 9 else 1)
        self.data['PPM'] = self.data['PPM'].apply(lambda x: 0 if x == 'N' else 1)
        self.data['super_conforming'] = self.data['super_conforming'].apply(lambda x: 1 if x == 'Y' else 0)
        self.data['pre-harp_sequence_number'] = self.data['pre-harp_sequence_number'].apply(lambda x: "" if pd.isna(x) else x)
        self.data['orig_interest_rate'] = self.data['orig_interest_rate'].apply(lambda x: 0.0 if x == '.' else float(x))
        self.data['MI_%'] = self.data['MI_%'].apply(lambda x: 0 if x == '.' else int(x))

        self.data['modification'] = self.data['modification'].apply(lambda x: 0 if x != 'Y' else 1)
        self.data['step_modification'] = self.data['step_modification'].apply(lambda x: 0 if x != 'Y' else 1)
        self.data['deferred_payment_plan'] = self.data['deferred_payment_plan'].apply(lambda x: 0 if x != 'Y' else 1)
        self.data['borrower_assistance_status'] = self.data['borrower_assistance_status'].apply(lambda x: 0 if x not in ['F', 'R', 'T'] else 1)
        self.data['current_deffered_UPB'] = self.data['current_deffered_UPB'].apply(lambda x: 0.0 if x == '.' else float(x))
        self.data['zero_balance_code'] = self.data['zero_balance_code'].apply(lambda x: int(1) if x == 1 else int(0))
        self.data['zero_balance_date'] = self.data['zero_balance_date'].apply(lambda x: 1 if pd.isna(x) else int(x))
        self.data['DDLPI'] = self.data['DDLPI'].apply(lambda x: 1 if pd.isna(x) else int(x))
        self.data['repurchase'] = self.data['repurchase'].apply(lambda x: 1 if x == 'Y' else 0)
        self.data['MI_recoveries'] = self.data['MI_recoveries'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['net_sales_proceeds'] = self.data['net_sales_proceeds'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['non_MI_recoveries'] = self.data['non_MI_recoveries'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['expenses'] = self.data['expenses'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['legal_costs'] = self.data['legal_costs'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['maintenance_costs'] = self.data['maintenance_costs'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['taxes_and_insurence'] = self.data['taxes_and_insurence'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['miscellaneous_expenses'] = self.data['miscellaneous_expenses'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['actual_loss'] = self.data['actual_loss'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['modification_cost'] = self.data['modification_cost'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['estimated_LTV'] = self.data['estimated_LTV'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['zero_balance_removal_UPB'] = self.data['zero_balance_removal_UPB'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['dlq_accrued_interest'] = self.data['dlq_accrued_interest'].apply(lambda x: 0.0 if pd.isna(x) else x)
        self.data['dlqc_due_to_disaster'] = self.data['dlqc_due_to_disaster'].apply(lambda x: 1 if x == 'Y' else 0)
        self.data['month'] = self.data['report_month'] % 100

        self.data = self.data[self.data.hpa.isnull() == False]

    def get_dataset(self, extra_cols=None):
        self.data[self.categorical_cols] = self.data[self.categorical_cols].astype('category')
        df_dummy = pd.get_dummies(self.data[self.categorical_cols])
        df = pd.concat([self.data[self.numerical_cols], df_dummy], axis=1)
        if extra_cols is not None:
            df = pd.concat([df, self.data[['report_month', 'zero_balance_removal_UPB']]], axis=1)
        return df

    def dataset_name(self):
        name = []
        for root, dirs, _ in os.walk(self.sll_dataset_dir):
            for dir_name in _:
                #print(dir_name[0:-4])
                name.append(dir_name[0:-4])
        return name[0]

    def get_cols(self, columns):
        return pd.DataFrame(self.data[columns], columns=columns)

    def _load_issuance_data(self, file_path):
        issuance = pd.read_csv(file_path, delimiter='|', header=None)
        issuance.columns = self._issuance_cols
        self.data.join(issuance.set_index('loan'))


In [ ]:

    # Visualize the dataset using PCA

sll_data_parser = StandardLoanLevelDatasetParser(max_rows_per_quarter=50000,
                                                  rows_to_sample=50000,
                                                  seed=2022)



# Loading data into dataframe

In [ ]:
sll_data_parser.load()

Loading Standard Loan-Level Dataset at path /content/drive/MyDrive/MBSPrepaymentPrediction/Datasets/StandardLoanLevelDataset/Data/
Adding data from historical_data_time_2010Q2.txt to dataset...
Adding data from historical_data_time_2010Q3.txt to dataset...
Adding data from historical_data_time_2010Q4.txt to dataset...
Adding data from historical_data_time_2011Q1.txt to dataset...
Adding data from historical_data_time_2011Q2.txt to dataset...
Adding data from historical_data_time_2011Q3.txt to dataset...
Adding data from historical_data_time_2011Q4.txt to dataset...
Adding data from historical_data_time_2012Q1.txt to dataset...
Adding data from historical_data_time_2012Q2.txt to dataset...
Adding data from historical_data_time_2012Q3.txt to dataset...
Adding data from historical_data_time_2012Q4.txt to dataset...
Adding data from historical_data_time_2013Q1.txt to dataset...
Adding data from historical_data_time_2013Q2.txt to dataset...
Adding data from historical_data_time_2013Q3.txt t

# Extracting dataframe and cleaning

In [ ]:
df_new =sll_data_parser.data

In [ ]:
drop_cols= ['current_loan_dlqc_status', 'repurchase', 'modification', 'zero_balance_code', 'zero_balance_date',
            'current_deffered_UPB', 'DDLPI', 'MI_recoveries', 'net_sales_proceeds', 'non_MI_recoveries', 'expenses',
            'legal_costs', 'maintenance_costs', 'taxes_and_insurence', 'miscellaneous_expenses', 'actual_loss',
            'modification_cost', 'step_modification', 'deferred_payment_plan', 'zero_balance_removal_UPB',
            'dlq_accrued_interest', 'dlqc_due_to_disaster', 'borrower_assistance_status', 'first_time_homebuyer_flag',
            'MSA', 'MI_%', 'number_of_units', 'orig_CLTV', 'channel', 'PPM', 'amortization_type', 'property_type',
            'postal_code', 'orig_loan_term', 'seller_name', 'servicer_name', 'super_conforming', 'pre-harp_sequence_number',
            'program_indicator', 'harp_indicator', 'property_valuation_method', 'io_indicator', 
            'estimated_LTV', 'report_quarter','first_payment_date', 'maturity_date']


df_new = df_new.drop(columns=drop_cols).drop_duplicates(keep='first') # removing duplicate loans

In [ ]:
# Testing to see if there are any nulls in the dataframe
df_new.isnull().sum()

loan_sequence_number     0
report_month             0
current_UPB              0
loan_age                 0
months_to_maturity       0
current_interest_rate    0
credit_score             0
occupancy_status         0
orig_DTI                 0
orig_UPB                 0
orig_LTV                 0
orig_interest_rate       0
property_state           0
loan_purpose             0
number_of_borrowers      0
year                     0
quarter                  0
report_year              0
hpi                      0
hpa                      0
mtg_rate                 0
unemp_rate               0
month                    0
dtype: int64

## Region identifier

In [ ]:
Region = {
    'West'      : ['AZ', 'CO', 'ID','NM', 'MT', 'UT', 'NV', 'WY', 'WA', 'AK', 'HI', 'OR', 'CA'],
    'Midwest'  : ['IN', 'IL', 'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'],
    'Northeast' : ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA'],
    'South'     : ['DE', 'DC', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV', 'AL', 'KY',
                   'MS', 'TN', 'AR', 'LA', 'OK', 'TX', 'PR']}





In [ ]:
def reverse_search_from_dictionary(dictionary, value):
    keys = dictionary.keys()
    for key in keys:
        if value in dictionary.get(key):
            return key
    return "Not present"



In [ ]:
df_new['region'] = df_new.apply(lambda row: reverse_search_from_dictionary(Region ,row.property_state), axis=1)

# Pool creation

## Pool specifications

In [ ]:
fico_list = [[301, 500], 
             #[401, 500], 
             [501, 700], 
             #[601, 700], 
             #[701, 850], 
             [701, 850]] 
interest_rate_list = [[0, 4.5-0.1e-3],
                      [5, 5.5-0.1e-3], 
                      [5.5, 6.5-0.1e-3], 
                      [6.5, 7.5-0.1e-3], 
                      [7.5, 10-0.1e-3]]

loan_list = [[0, 150e3-1], 
            #[85e3, 150e3-1], 
            [150e3, 350e3-1], 
            # [250e3, 350e3-1],
            [350e3, 85000e3-1]]

region_list = ['Northeast', 'West', 'Midwest', 'South']

year_list =[2011,
            2012,
            2013,
            2014,
            2015,
            2016,
            2017,
            2018,
            2019,
            2020,
            2021]

In [ ]:
populated_list = []
i = 0
for score in fico_list:
  for interest in interest_rate_list:
    for loan in loan_list:
      for region in region_list:
        for year in year_list:
        #print(i)
          populated_list.append([i , score, interest, loan, region, year])
          i +=1

Setting up the conditions for the pools

In [ ]:
# create a list of our conditions
poolid = []
conditions = []
for value in populated_list:
      poolid.append(value[0])

for value in populated_list:
  conditions.append((df_new['credit_score'] > value[1][0]) & 
                    (df_new['credit_score'] <= value[1][1]) & 
                    (df_new['current_interest_rate'] > value[2][0]) & 
                    (df_new['current_interest_rate'] <= value[2][1]) &
                    (df_new['current_UPB'] > value[3][0]) &
                    (df_new['current_UPB'] <= value[3][1]) &
                    (df_new['region'] == value[4])&
                    (df_new['year'] == value[5]))






Creating a copy to ensure that we can go back if data becomes fucked up

In [ ]:
df_new_backup = df_new.copy()

In [ ]:
# create a list of the values we want to assign for each condition

# create a new column and use np.select to assign values to it using our lists as arguments
df_new_backup['poolid'] = np.select(conditions, poolid)

# display updated DataFrame
df_new_backup

,loan_sequence_number,report_month,current_UPB,loan_age,months_to_maturity,current_interest_rate,credit_score,occupancy_status,orig_DTI,orig_UPB,...,year,quarter,report_year,hpi,hpa,mtg_rate,unemp_rate,month,region,poolid
0,F10Q20000001,201005,244000.00,0,360,5.000,753,P,999,244000,...,2010,2,2010,313.78,-4.532746,4.78,9.6,5,Midwest,0
1,F10Q20000001,201006,244000.00,1,359,5.000,753,P,999,244000,...,2010,2,2010,313.78,-4.532746,4.69,9.4,6,Midwest,0
2,F10Q20000001,201007,244000.00,2,358,5.000,753,P,999,244000,...,2010,2,2010,313.78,-4.532746,4.54,9.4,7,Midwest,0
3,F10Q20000001,201008,243000.00,3,357,5.000,753,P,999,244000,...,2010,2,2010,317.06,4.247293,4.36,9.5,8,Midwest,0
4,F10Q20000001,201009,243000.00,4,356,5.000,753,P,999,244000,...,2010,2,2010,317.06,4.247293,4.32,9.5,9,Midwest,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,F10Q10000766,201102,261001.39,12,348,4.875,799,P,14,265000,...,2010,1,2011,295.27,-8.559801,4.95,9.0,2,Midwest,0
49996,F10Q10000766,201103,260659.31,13,347,4.875,799,P,14,265000,...,2010,1,2011,295.27,-8.559801,4.86,9.0,3,Midwest,0
49997,F10Q10000766,201104,260315.84,14,346,4.875,799,P,14,265000,...,2010,1,2011,290.39,-6.448807,4.78,9.1,4,Midwest,0
49998,F10Q10000766,201105,259970.97,15,345,4.875,799,P,14,265000,...,2010,1,2011,290.39,-6.448807,4.60,9.0,5,Midwest,0


In [ ]:
df_new_backup = df_new_backup.loc[df_new_backup['poolid'] != 0]  # hvordan slipper jeg uden at skulle fjerne en hel del observationer

# Doing payment variables

Shifting the current unpaid balance

In [ ]:
df_new_backup['current_UPB_shifted'] = df_new_backup.groupby(['loan_sequence_number'])['current_UPB'].shift(1)

In [ ]:
df_new_backup = df_new_backup.dropna()

## Payment Actual

Her skal der laves følgende beregning 



PP^ACT = \P_{t-1)-P_t\

Dog skal det sker for hvert individuelt låneneummer

Her er P = current_UPB  (current unpaid balance)





In [ ]:
df_new_backup['pp_act'] =  df_new_backup['current_UPB_shifted'] - df_new_backup['current_UPB']
#df_new = df_new.loc[df_new['pp_act'] >= 0] # adjustment due to data strcuture, which would yield high negative values when loans was repaid in full
df_new_backup = df_new_backup.dropna()


Checking the dataset

In [ ]:
df_new_backup.loc[df_new_backup['pp_act'] != 0]

,loan_sequence_number,report_month,current_UPB,loan_age,months_to_maturity,current_interest_rate,credit_score,occupancy_status,orig_DTI,orig_UPB,...,report_year,hpi,hpa,mtg_rate,unemp_rate,month,region,poolid,current_UPB_shifted,pp_act
1181,F11Q10000022,201110,44591.92,7,353,5.750,796,I,37,45000,...,2011,258.88,2.367550,4.10,8.8,10,South,1617,45000.00,408.08
1182,F11Q10000022,201111,44543.42,8,352,5.750,796,I,37,45000,...,2011,258.88,2.367550,3.98,8.6,11,South,1617,44591.92,48.50
1183,F11Q10000022,201112,44494.69,9,351,5.750,796,I,37,45000,...,2011,258.35,-0.816401,3.95,8.5,12,South,1617,44543.42,48.73
1184,F11Q10000022,201201,44445.72,10,350,5.750,796,I,37,45000,...,2012,251.69,-9.919668,3.98,8.3,1,South,1617,44494.69,48.97
1185,F11Q10000022,201202,44396.52,11,349,5.750,796,I,37,45000,...,2012,251.69,-9.919668,3.95,8.3,2,South,1617,44445.72,49.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49552,F20Q40003695,202012,216000.00,1,359,2.500,808,P,21,217000,...,2020,721.76,10.510612,2.67,6.7,12,West,1384,217000.00,1000.00
49647,F20Q40003702,202012,297000.00,1,359,2.990,741,P,25,298000,...,2020,626.57,14.390318,2.67,6.7,12,Northeast,1373,298000.00,1000.00
49697,F20Q40003706,202012,338000.00,1,359,3.000,788,P,31,339000,...,2020,630.06,10.786383,2.67,6.7,12,West,1384,339000.00,1000.00
49778,F20Q40003712,202012,387000.00,1,359,2.875,756,P,48,388000,...,2020,394.36,7.671125,2.67,6.7,12,Midwest,1439,388000.00,1000.00


## Expected payments
beskrivelsen kan findes her [Expcted payments](https://ibb.co/yy5gscd)

In [ ]:
# Expected prepayments
#Annuity payment
import numpy as np

def AP(P0, r, n):
      rate_pct = r/100
      periods = n
      temp1 = rate_pct*P0
      temp2 = 1-np.power(1+rate_pct, -periods)

      return temp1/temp2

def pp_exp(P0, Pt, r, n):
     return AP(P0, r, n) - (r/100 * Pt)

df_new_backup['pp_ext'] = pp_exp(df_new_backup['orig_UPB'],df_new_backup['current_UPB_shifted'], df_new_backup['current_interest_rate'], df_new_backup['months_to_maturity'])

In [ ]:
df_new_backup['prp'] = df_new_backup['pp_act'] - df_new_backup['pp_ext']

In [ ]:
df_new_backup = df_new_backup.dropna() # dropping nans

## Single Month Mortality

Creating a sum of prp

In [ ]:
df_new_backup['prp_sum'] = df_new_backup.groupby(['poolid', 'report_year', 'month'])['prp'].transform('sum')

Creating a sum of past unpaid balance

In [ ]:
df_new_backup['UBP_shift_sum'] = df_new_backup.groupby(['poolid', 'report_year', 'month'])['current_UPB_shifted'].transform('sum')

Calcuating SMM

In [ ]:
df_new_backup['SMM'] = (df_new_backup['prp_sum'] / df_new_backup['UBP_shift_sum']).clip(lower=0) # SMM

Doing CPR calculations

In [ ]:
df_new_backup['CPR']  = 1-np.power(1-df_new_backup['SMM'], 12)

Should be good here, lets see the dataframe

In [ ]:
df_new_backup.loc[df_new_backup['loan_sequence_number'] == 'F11Q10000022'].groupby(['loan_sequence_number','property_state', 'report_month'])['hpi'].mean()

loan_sequence_number  property_state  report_month
F11Q10000022          GA              201104          257.37
                                      201105          257.37
                                      201106          257.37
                                      201107          257.37
                                      201108          258.88
                                                       ...  
                                      201706          333.44
                                      201707          333.44
                                      201708          338.75
                                      201709          338.75
                                      201710          338.75
Name: hpi, Length: 79, dtype: float64

# Variable Creation

## Interest rate Incentive

In [ ]:
#interest rate incentive
def interest_incentive(current_rate, market_rate):
    return current_rate-market_rate

#
df_new_backup['incentive'] = interest_incentive(df_new_backup['mtg_rate'], df_new_backup['current_interest_rate'])


## Rolling Interest rate Incentive

In [ ]:
df_new_backup['rolling'] = df_new_backup.groupby(['loan_sequence_number'])['incentive'].transform(lambda x: x.rolling(24, 1).mean())

## Spread at orgination

In [ ]:
def SATO(market_rate, orginal_rate):
    return orginal_rate-market_rate
df_new_backup['SATO'] = SATO(df_new_backup['orig_interest_rate'], df_new_backup['mtg_rate'])


## HPI change

In [ ]:
df_new_backup['hpi_0'] =df_new_backup.groupby('loan_sequence_number')['hpi'].transform('first')

In [ ]:
df_new_backup['hpi_pct'] = (df_new_backup['hpi'] / df_new_backup['hpi_0'])-1

## Cos and Sin Months

In [ ]:
import numpy as np
df_new_backup['sin_month'] = np.sin(2 * np.pi * df_new_backup['month'].astype(int)*30/360)
df_new_backup['cos_month'] = np.cos(2 * np.pi * df_new_backup['month'].astype(int)*30/360)


## Standard Deviation of Unpaid Balance

In [ ]:
df_new_backup['UPB_std'] = df_new_backup.groupby(['loan_sequence_number'])['current_UPB'].transform('std')

## Standard Deviation of Loan Size

In [ ]:
df_new_backup['orig_UPB_std'] = df_new_backup.groupby(['loan_sequence_number'])['orig_UPB'].transform('std')

# Weigted Average Code

Creating a weighted average function

In [ ]:
def weigted_average(data, key):
  temp_df = data[['loan_sequence_number', 'poolid', 'report_year', 'month']]
  temp_df['pool_balance'] = data.groupby(['poolid', 'report_year', 'month'])['current_UPB'].transform('sum')
  temp_df['wols_P'] =  data[key] * (data['current_UPB']/temp_df['pool_balance'])
  temp_df['wa_'+key] = temp_df.groupby(['poolid', 'report_year', 'month'])['wols_P'].transform('sum')
  #temp_df.append(temp_df['wa_'+key])
  return temp_df['wa_'+key]


Adding weigthed averages in a loop

In [ ]:
keys = ['current_UPB', 'loan_age', 'SATO', 'orig_DTI', 'orig_LTV', 'unemp_rate', 'orig_UPB', 'incentive', 'rolling', 'hpi_pct']

for key in keys:
  df_new_backup['wa_'+key] = weigted_average(df_new_backup, key)
  print(f'Weighted Average for {key} - Done')

Weighted Average for current_UPB - Done
Weighted Average for loan_age - Done
Weighted Average for SATO - Done
Weighted Average for orig_DTI - Done
Weighted Average for orig_LTV - Done
Weighted Average for unemp_rate - Done
Weighted Average for orig_UPB - Done
Weighted Average for incentive - Done
Weighted Average for rolling - Done
Weighted Average for hpi_pct - Done


Lets check the dataframe output

In [ ]:
df_new_backup#.loc[(df_new_backup['poolid'] == 1617) & (df_new_backup['report_year'] == 2020) & (df_new_backup['month'] == 12)]

,loan_sequence_number,report_month,current_UPB,loan_age,months_to_maturity,current_interest_rate,credit_score,occupancy_status,orig_DTI,orig_UPB,...,wa_current_UPB,wa_loan_age,wa_SATO,wa_orig_DTI,wa_orig_LTV,wa_unemp_rate,wa_orig_UPB,wa_incentive,wa_rolling,wa_hpi_pct
1175,F11Q10000022,201104,45000.0,1,359,5.750,796,I,37,45000,...,56818.181818,1.590909,-0.970000,35.818182,69.090909,9.1,56818.181818,-0.970000,-0.946364,-0.025148
1176,F11Q10000022,201105,45000.0,2,358,5.750,796,I,37,45000,...,56818.181818,2.590909,-1.150000,35.818182,69.090909,9.0,56818.181818,-1.150000,-1.026515,-0.025148
1177,F11Q10000022,201106,45000.0,3,357,5.750,796,I,37,45000,...,67572.916667,2.484375,-1.240000,33.760417,71.614583,9.1,67572.916667,-1.240000,-1.151784,-0.014408
1178,F11Q10000022,201107,45000.0,4,356,5.750,796,I,37,45000,...,67170.542636,2.848837,-1.200000,31.263566,71.201550,9.0,67170.542636,-1.200000,-1.168178,-0.010722
1179,F11Q10000022,201108,45000.0,5,355,5.750,796,I,37,45000,...,66797.665370,3.852140,-1.530000,31.264591,71.186770,9.0,67112.840467,-1.530000,-1.275123,-0.004960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49984,F20Q40003727,202012,93000.0,1,359,3.000,775,P,25,93000,...,111267.718376,5.215721,-0.625927,30.738850,72.066554,6.7,114741.071259,-0.625927,-0.477682,0.022345
29505,F21Q30006434,202109,67000.0,1,359,3.875,706,P,38,67000,...,67000.000000,1.000000,-0.865000,38.000000,80.000000,4.7,67000.000000,-0.865000,-0.865000,0.000000
29506,F21Q30006434,202110,67000.0,2,358,3.875,706,P,38,67000,...,67000.000000,2.000000,-0.735000,38.000000,80.000000,4.6,67000.000000,-0.735000,-0.800000,0.000000
29507,F21Q30006434,202111,67000.0,3,357,3.875,706,P,38,67000,...,67000.000000,3.000000,-0.775000,38.000000,80.000000,4.2,67000.000000,-0.775000,-0.791667,0.000000


### Working weigthed average approach

Shit works, correct balance based on poolid and month

In [ ]:
# df_new_backup['pool_balance'] = df_new_backup.groupby(['poolid', 'report_year', 'month'])['current_UPB'].transform('sum')

Now we create weights based on the current outstanding unpaid balance

In [ ]:
# df_new_backup['weighted_P'] = (df_new_backup['current_UPB']/df_new_backup['pool_balance'])

Checking to see how the weight column turned out

In [ ]:
# df_new_backup.groupby(['poolid', 'report_year', 'month'])['weighted_P'].sum()

Seems fine, but need to make a check to see if there are values not equal to 1

In [ ]:
# df_new_backup.columns

Create

In [ ]:
# df = df_new_backup[['loan_sequence_number', 'poolid', 'report_year', 'month', 'current_interest_rate']]

In [ ]:
# df['wols_P'] = df_new_backup['current_interest_rate'] * (df_new_backup['current_UPB']/df_new_backup['pool_balance'])

In [ ]:
# df

In [ ]:
# df['wa_current_interest_rate'] = df.groupby(['poolid', 'report_year', 'month'])['wols_P'].transform('sum')

In [ ]:
# df.loc[df['poolid'] == 1617].to_excel('test.xlsx')

In [ ]:
# df.loc[(df['poolid'] == 1617) & (df['report_year'] == 2020) & (df['month'] == 12)]

# Weight calcuation working | create the new dataframe in a loop and we are good to go

# Exporting dataset

In [ ]:
# # Write df into a sequence of small files

# #out_dir = '../30y fixed rate data/fnm/conforming/clean data'
# out_dir = '../30y fixed rate data/fnm/jumbos/clean data'

# print(f'df.shape = {df.shape}')

# rows = df.shape[0]

# N = rows//30000 + 1

# for i in range(N):
#     df.iloc[i*30000:(i+1)*30000,].to_csv(f'{out_dir}/pools_part{i+1}.csv',index=False)